In [28]:
!pip3 install groq

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [29]:
import requests
import pandas as pd
import re
from datasets import load_dataset
import os


df = load_dataset("ChilleD/SVAMP")


In [30]:
test_df = df['test'].to_pandas()

In [31]:
# test_df = test_df.head(300)

In [32]:
test_df

,ID,Body,Question,Equation,Answer,Type,question_concat
0,chal-736,Winter is almost here and most animals are mig...,How many more bird families flew away to afric...,( 62.0 - 35.0 ),27,Subtraction,Winter is almost here and most animals are mig...
1,chal-162,Paige raised 7 goldfish and 12 catfish in the ...,How many fishes disappeared?,( ( 7.0 + 12.0 ) - 15.0 ),4,Subtraction,Paige raised 7 goldfish and 12 catfish in the ...
2,chal-349,Marco and his dad went strawberry picking. Tog...,How much did his dad's strawberries weigh now?,( ( 22.0 - 36.0 ) + 30.0 ),16,Addition,Marco and his dad went strawberry picking. Tog...
3,chal-390,Debby bought 200 water bottles and 256 soda bo...,How many days would the soda bottles last?,( 256.0 / 4.0 ),64,Common-Division,Debby bought 200 water bottles and 256 soda bo...
4,chal-781,There were 106 dollars in Olivia's wallet. Aft...,How much did she spend at the supermarket?,( ( 106.0 - 26.0 ) - 49.0 ),31,Subtraction,There were 106 dollars in Olivia's wallet. Aft...
...,...,...,...,...,...,...,...
295,chal-143,"7 red peaches, 15 yellow peaches and 8 green p...",How many peaches are in the basket?,( ( 7.0 + 15.0 ) + 8.0 ),30,Addition,"7 red peaches, 15 yellow peaches and 8 green p..."
296,chal-73,Jack received 3 emails and 64 letters in the m...,How many emails did jack receive in the day?,( 3.0 + 5.0 ),8,Addition,Jack received 3 emails and 64 letters in the m...
297,chal-471,Ed had 2 more marbles than Doug. Doug lost som...,How many marbles did Doug lose?,( 19.0 - 2.0 ),17,Subtraction,Ed had 2 more marbles than Doug. Doug lost som...
298,chal-542,Dan has $ 3 left with him after he bought a ca...,How much money did he have initially?,( 3.0 + 2.0 ),5,Addition,Dan has $ 3 left with him after he bought a ca...


In [33]:
correct_labels = []
for i in range(len(test_df['Answer'])):
    correct_labels.append(test_df['Answer'][i])

In [34]:
zero_shot_prompts = []
chain_of_thought_prompts = []
for i in range(len(test_df)):
    question = test_df['question_concat'][i]
    answer = correct_labels[i]
    prompt = question
    zero_shot_prompts.append(prompt)

In [35]:
zero_shot_prompts[1]

'Paige raised 7 goldfish and 12 catfish in the pond but stray cats loved eating them. Now she has 15 left. How many fishes disappeared?'

In [36]:
os.environ['GROQ_API_KEY']='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'

In [37]:
from groq import Groq
client=Groq()

In [38]:
def answer(query):
    system_prompt = {"role": "system", "content": "You will be given a math word problem, your job is to solve this problem and return one numerical answer. The template to answer the query is to have the last line of the output as Answer: <<<Numerical Answer>>>"}
    chat_history = [system_prompt]
    chat_history.append({"role": "user", "content": query})
    response = client.chat.completions.create(model="llama-3.1-8b-instant", messages=chat_history, temperature=0)
    return response.choices[0].message.content

In [39]:
def extract_answer_value(response):
    match = re.search(r'Answer:\s*[^0-9]*([\d]+)', response)
    if match:
        try:
            return int(match.group(1)) 
        except ValueError:
            return "X"
    return "X"  

In [40]:
bad = 0
predictions_zeroshot_model = []
for i in range(len(zero_shot_prompts)):
    try:
        answerrr = answer(zero_shot_prompts[i])
        extracted = extract_answer_value(answerrr)
    except:
        extracted = "X"
    if(extracted == "X"):
        bad += 1
    predictions_zeroshot_model.append(extracted)
    if(i%30 == 0):
        print(i)
        print(extracted)
        print("---------------")
print("Bad Responses:",bad)

0
27
---------------
30
3
---------------
60
128
---------------
90
28
---------------
120
4
---------------
150
1
---------------
180
3
---------------
210
3
---------------
240
2226
---------------
270
12
---------------
Bad Responses: 14


In [46]:
predictions_zeroshot_model[:10]

[27, 4, 16, 64, 31, 720, 21, 64, 720, 143550]

In [47]:
correct_labels[:10]

['27', '4', '16', '64', '31', '720', '21', '64', '450', '143550']

In [48]:
bad

14

In [49]:
def accuracy(predictions,labels):
    correct = 0
    for i in range(len(labels)):
        if(labels[i] == str(predictions[i])):
            correct+=1
    return correct*100/len(predictions)

In [50]:
print(accuracy(predictions_zeroshot_model,correct_labels))

86.33333333333333
